Evaluation

Import environment variables

In [11]:
import dotenv
import os

dotenv.load_dotenv()


True

Load docs

In [118]:
from langchain_community.document_loaders import DirectoryLoader
from langchain_community.document_loaders import PyMuPDFLoader


path = "test_data/"
loader = DirectoryLoader(path, glob="*.pdf", loader_cls=PyMuPDFLoader)
docs = loader.load()

docs[0]

Document(metadata={'producer': 'Microsoft: Print To PDF', 'creator': '', 'creationdate': '2023-07-15T11:40:07+05:30', 'source': 'test_data/INTRODUCTION-TO-RDBMS.pdf', 'file_path': 'test_data/INTRODUCTION-TO-RDBMS.pdf', 'total_pages': 24, 'format': 'PDF 1.7', 'title': 'Microsoft Word - Introduction to relational databases', 'author': 'Administrator', 'subject': '', 'keywords': '', 'moddate': '2023-07-15T11:40:07+05:30', 'trapped': '', 'modDate': "D:20230715114007+05'30'", 'creationDate': "D:20230715114007+05'30'", 'page': 0}, page_content='Introduction to relational databases \n \nIntroduction to relational databases \n \n\uf0b7 A relational database is based on the relational model and uses a collection of \ntables to represent both data and the relationship among those data. \n\uf0b7 It also includes DDL and DML  \n\uf0b7 Relational database was originally defined by Edger Codd at IBM Research \ncenter in 1970. \n\uf0b7 In relational database, user only needs to understand logical str

choose Llm

In [119]:
from ragas.llms import LangchainLLMWrapper
from langchain_openai import ChatOpenAI
from ragas.embeddings import OpenAIEmbeddings
import openai

evaluator_llm = evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
openai_client = openai.OpenAI()
generator_embeddings = OpenAIEmbeddings(client=openai_client)

/tmp/ipykernel_82574/2774485547.py:6: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  evaluator_llm = evaluator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))
/tmp/ipykernel_82574/2774485547.py:7: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use the modern LLM providers instead: from ragas.llms.base import llm_factory; llm = llm_factory('gpt-4o-mini') or from ragas.llms.base import instructor_llm_factory; llm = instructor_llm_factory('openai', client=openai_client)
  generator_llm = LangchainLLMWrapper(ChatOpenAI(model="gpt-4o-mini"))


Generate testset

In [120]:
from ragas.testset import TestsetGenerator

generator = TestsetGenerator(llm=generator_llm, embedding_model=generator_embeddings)
dataset = generator.generate_with_langchain_docs(docs, testset_size=5)

Applying SummaryExtractor:   0%|          | 0/23 [00:00<?, ?it/s]

Applying EmbeddingExtractor:   0%|          | 0/23 [00:00<?, ?it/s]/root/py/proj/backend/.venv/lib/python3.13/site-packages/ragas/testset/transforms/base.py:188: UserWarning: Using sync embedding model OpenAIEmbeddings in async context. This may impact performance. Consider using an async-compatible embedding model for better performance.
  property_name, property_value = await self.extract(node)
Generating Samples: 100%|██████████| 6/6 [00:03<00:00,  1.59it/s]


In [121]:
dataset.to_pandas()

,user_input,reference_contexts,reference,synthesizer_name
0,Who originally defined the relational database...,[Introduction to relational databases \n \nInt...,The relational database was originally defined...,single_hop_specific_query_synthesizer
1,What is the role of a primary key in a relatio...,[ A stored procedure is a high-end database t...,A primary key is a field that uniquely identif...,single_hop_specific_query_synthesizer
2,What role do stored procedures play in relatio...,[<1-hop>\n\n A stored procedure is a high-end...,Stored procedures are high-end database tools ...,multi_hop_abstract_query_synthesizer
3,What is the significance of using a full outer...,[<1-hop>\n\nName \nTamil \nSelva \nDurai \nKut...,The significance of using a full outer join in...,multi_hop_abstract_query_synthesizer
4,What is the relation between the tables Deposi...,[<1-hop>\n\n Consider two table (Depositor & ...,The relation between the tables Depositor and ...,multi_hop_specific_query_synthesizer
5,What is the purpose of using embedded SQL in p...,[<1-hop>\n\nAdvanced SQL features \nEmbedded S...,The purpose of using embedded SQL in programmi...,multi_hop_specific_query_synthesizer


Prepere data

In [122]:
from agent.agent import tutor_agent
from langchain_core.messages import HumanMessage
import json


In [123]:
#THIS CELL IS FOR DEBUGGING

# from agent.utils.tools import read_from_temp

# eval_contexts = read_from_temp()
# eval_contexts

import json

res = tutor_agent.invoke({"messages": [HumanMessage("How to use key?")], "retrieved_docs": []})
msgs = res["messages"]

retrieved_contexts = []

for msg in msgs:
    if msg.name == "knowledge_base_search":
        retrieved_contexts.append(msg.content)

parsed_contents = [json.loads(item) for item in retrieved_contexts]

contents = []
for inner_list in parsed_contents:
    for item in inner_list:
        contents.append(item)

len(contents)

# res["retrieved_docs"]


5

In [124]:
for test_row in dataset:
    query = test_row.eval_sample.user_input
    response = tutor_agent.invoke({"messages": [HumanMessage(query)]})

    msgs = response["messages"]

    retrieved_contexts = []

    for msg in msgs:
        if msg.name == "knowledge_base_search":
            retrieved_contexts.append(msg.content)

    parsed_contents = [json.loads(item) for item in retrieved_contexts]

    contents = []
    for inner_list in parsed_contents:
        for item in inner_list:
            contents.append(item)

    test_row.eval_sample.retrieved_contexts = contents
    test_row.eval_sample.response = response["messages"][-1].content
    

In [125]:
from ragas import EvaluationDataset

evaluation_dataset = EvaluationDataset.from_pandas(dataset.to_pandas())

Run evaluation

In [126]:
from ragas import evaluate
from ragas.llms import LangchainLLMWrapper
from ragas.metrics import Faithfulness, ResponseRelevancy, LLMContextPrecisionWithReference, LLMContextRecall

result = evaluate(dataset=evaluation_dataset,metrics=[Faithfulness(), ResponseRelevancy(), LLMContextPrecisionWithReference(), LLMContextRecall()],llm=evaluator_llm)
result

Evaluating:   4%|▍         | 1/24 [00:01<00:37,  1.63s/it]LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
LLM returned 1 generations instead of requested 3. Proceeding with 1 generations.
Evaluating: 100%|██████████| 24/24 [00:36<00:00,  1.51s/it]


{'faithfulness': 0.7728, 'answer_relevancy': 0.9538, 'llm_context_precision_with_reference': 0.8448, 'context_recall': 0.9250}

## Observation

The system performed 'well' overall.

scores: {
    'faithfulness': 0.7742, 
    'answer_relevancy': 0.9451, 
    'llm_context_precision_with_reference': 0.9021, 
    'context_recall': 0.9444
}

Faithfulness is 'good' but stands out from other values and can be improved. One way to improve it to add Rerenker